In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import zarr
import os 
from Compressor import Focalizer
import pandas as pd
from pathlib import Path 
# Specify device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    
input_tensor = torch.load('/home/roberto/PythonProjects/SSFocus/Data/4096_test_fft2D.pt').to(device)

# Usage example
# file_path = "Mini_R2F.zarr"

# # To read the root array or group
# root = read_zarr_file(file_path)

# # To read a specific array or group
# raw = read_zarr_file(file_path, "raw")
# gt = read_zarr_file(file_path, "gt")

%load_ext autoreload
%autoreload 2

In [2]:
# Load model and weights
folders = [x.stem.split('_')[-1] for x in Path('/home/roberto/PythonProjects/SSFocus/lightning_logs').iterdir()]
idx = max([int(x) for x in folders])
version = f'version_{idx}'
print('Loaded the version:', version)

checkpoint_file = find_checkpoint(f"/home/roberto/PythonProjects/SSFocus/lightning_logs/{version}/")
checkpoint = torch.load(checkpoint_file, map_location=device)

aux = pd.read_pickle('/home/roberto/PythonProjects/SSFocus/Data/RAW/SM/numpy/s1a-s1-raw-s-vv-20200509t183227-20200509t183238-032492-03c34a_pkt_8_metadata.pkl')
eph = pd.read_pickle('/home/roberto/PythonProjects/SSFocus/Data/RAW/SM/numpy/s1a-s1-raw-s-vv-20200509t183227-20200509t183238-032492-03c34a_ephemeris.pkl')
model = Focalizer(metadata={'aux':aux, 'ephemeris':eph}).to(device)

# If your model was wrapped with a LightningModule, you may need to prepend 'model.' to each key
model.load_state_dict({key.replace('model.', ''): value for key, value in checkpoint['state_dict'].items()})
model.eval()  # Set the model to evaluation mode


Loaded the version: version_8
Len tx replica: 4549


RuntimeError: The expanded size of the tensor (227) must match the existing size (4549) at non-singleton dimension 0.  Target sizes: [227].  Tensor sizes: [4549]

In [ ]:
# Perform inference
with torch.no_grad():
    output_tensor = model(input_tensor.unsqueeze(0).unsqueeze(0))  # Adding a batch dimension
    output_tensor = output_tensor.squeeze(0).squeeze(0)
    output_tensor = output_tensor.cpu().numpy()  # Removing batch dimension and sending to cpu
# model._plot_tensor(input_tensor)
model._plot_tensor(output_tensor)

In [ ]:
print(list(model.parameters()))

In [ ]:
model.V()

In [ ]:
# model._plot_rg_filter()